## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-04-16-57-09 +0000,nyt,Suspect Arrested in Inquiry Into Pipe Bombs in...,https://www.nytimes.com/2025/12/04/us/politics...
1,2025-12-04-16-55-36 +0000,nypost,Texas A&M cheerleader Brianna Aguilera’s death...,https://nypost.com/2025/12/04/us-news/tragic-t...
2,2025-12-04-16-55-31 +0000,bbc,UK and Norway form naval alliance to hunt Russ...,https://www.bbc.com/news/articles/c3v152ep9l0o...
3,2025-12-04-16-54-39 +0000,nypost,Eric Ripert’s Le Bernardin again ranked No. 1 ...,https://nypost.com/2025/12/04/business/eric-ri...
4,2025-12-04-16-49-02 +0000,bbc,Trump hosting signing of peace deal between le...,https://www.bbc.com/news/articles/cjrjn88jqn4o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2381/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,trump,54
149,new,18
109,putin,16
59,ukraine,14
373,he,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
234,2025-12-04-00-05-45 +0000,nyt,Trump Faces Choices on Russia-Ukraine Talks as...,https://www.nytimes.com/2025/12/03/us/politics...,103
159,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...,99
80,2025-12-04-13-29-44 +0000,wapo,Putin maintains hard line in talks as Trump sa...,https://www.washingtonpost.com/world/2025/12/0...,98
279,2025-12-03-22-33-23 +0000,bbc,Trump administration to roll back fuel economy...,https://www.bbc.com/news/articles/cly13we8zxeo...,92
169,2025-12-04-09-00-00 +0000,cbc,Trump offered Putin a tactical victory in Ukra...,https://www.cbc.ca/news/world/trump-putin-envo...,90


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
234,103,2025-12-04-00-05-45 +0000,nyt,Trump Faces Choices on Russia-Ukraine Talks as...,https://www.nytimes.com/2025/12/03/us/politics...
238,85,2025-12-04-00-00-00 +0000,wsj,Defense Secretary Pete Hegseth shocked officia...,https://www.wsj.com/politics/national-security...
39,61,2025-12-04-15-18-00 +0000,wsj,"New York Times Sues Hegseth, Defense Departmen...",https://www.wsj.com/business/media/new-york-ti...
347,49,2025-12-03-19-37-05 +0000,latimes,Former sheriff's deputy killed after police ch...,https://www.latimes.com/california/story/2025-...
116,49,2025-12-04-11-53-24 +0000,nypost,Admiral who ordered September boat strike will...,https://nypost.com/2025/12/04/us-news/admiral-...
47,46,2025-12-04-15-04-19 +0000,nypost,Putin’s alleged secret daughter tells Ukrainia...,https://nypost.com/2025/12/04/world-news/putin...
159,45,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...
128,44,2025-12-04-11-02-24 +0000,bbc,Israel says Palestinian groups returned body o...,https://www.bbc.com/news/articles/cd6x2j4g3z7o...
219,44,2025-12-04-01-55-16 +0000,nypost,Halle Berry bashes Gov. Gavin Newsom at NYT su...,https://nypost.com/2025/12/03/us-news/halle-be...
65,41,2025-12-04-14-09-00 +0000,wsj,The FBI arrested a Virginia man in connection ...,https://www.wsj.com/politics/national-security...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
